# FIT5149 S2 2020 
# Assessment 2:  Residential Energy Appliance Classification


Student information
    - Family Name: WANG
    - Given Name: ZHIYIN
- Student ID: 31436285
- Student email: zwan0252@student.monash.edu

Programming Language: R 4.0.5 in Jupyter Notebook

In [8]:
# import libraries
library(randomForest)
library(ROSE)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loaded ROSE 0.0-3




## 1. Import data

In [1]:
# import train and test data
train  <- read.csv("./train_data_withlabels.csv")
test <- read.csv("./test_data_nolabels.csv")

In [5]:
# the data set is highly imbalanced
table(train$ac)
table(train$ev)
table(train$oven)
table(train$wash)
table(train$dryer)


     0      1 
316521 101199 


     0      1 
415385   2335 


     0      1 
411764   5956 


     0      1 
409767   7953 


     0      1 
404144  13576 

## 2. Train classifier

In [6]:
# numeric target variable to factor 
train$ac <- as.factor(train$ac)
train$ev <- as.factor(train$ev)
train$oven <- as.factor(train$oven)
train$wash <- as.factor(train$wash)
train$dryer <- as.factor(train$dryer)

In [9]:
# generate under sampling data synthetically for each appliances, method = "under", N = number of postitive label * 2
ac_balanced_under <- ovun.sample(ac ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = train, method = "under", N = 101199*2, seed = 1)$data

ev_balanced_under <- ovun.sample(ev ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = train, method = "under", N = 2235*2, seed = 1)$data

oven_balanced_under <- ovun.sample(oven ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = train, method = "under", N = 5956*2, seed = 1)$data

wash_balanced_under <- ovun.sample(wash ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = train, method = "under", N = 7953*2, seed = 1)$data

dryer_balanced_under <- ovun.sample(dryer ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = train, method = "under", N = 13576*2, seed = 1)$data

### ac

In [10]:
# train random forest for ac
random.ac <- randomForest(ac ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = ac_balanced_under, ntree = 200, mtry=3, importance=TRUE)

### ev

In [11]:
# train random forest for ev
random.ev <- randomForest(ev ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = ev_balanced_under, ntree = 1000, mtry=3, importance=TRUE)

### oven

In [12]:
# train random forest for oven
random.oven <- randomForest(oven ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = oven_balanced_under, ntree = 1000, mtry=3, importance=TRUE)

### wash

In [13]:
# train random forest for wash
random.wash <- randomForest(wash ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = wash_balanced_under, ntree = 1000, mtry=3, importance=TRUE)

### dryer

In [14]:
# train random forest for dryer
random.dryer <- randomForest(dryer ~ load + hourofday + dif + absdif + max + var + entropy + 
                          nonlinear + hurst, data = dryer_balanced_under, ntree = 1000, mtry=3, importance=TRUE)

## 3. Prediction on test data

In [15]:
# predict labels on test data features and record the predictions
test.predict.ac <- predict(random.ac, test, "class")
test.predict.ev <- predict(random.ev, test, type="class")
test.predict.oven <- predict(random.oven, test, type="class")
test.predict.wash <- predict(random.wash, test, type="class")
test.predict.dryer <- predict(random.dryer, test, type="class")

## 4. Output to csv

In [17]:
# save all prediction into one dataframe with correct format
output <- data.frame(col_index = 1:105540, ac = test.predict.ac, ev = test.predict.ev, oven = test.predict.oven, 
                     wash = test.predict.wash, dryer = test.predict.dryer)

# save to local csv file
write.csv(output, "./pred_labels.csv", row.names = FALSE)